In [1]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
%run $header_path

C:\Users\raphael.wargon\Documents\GitHub\sorting_univ\script/headers/
initalizing spark nlp session w gpu


'C:\\Users\\raphael.wargon\\Documents\\GitHub\\sorting_univ\\script\\2_panel_creation'

In [22]:
info_status_path = os.path.abspath("") + '\\info_statuses.ipynb'
%run $info_status_path

In [2]:
df_inst = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'institutions.parquet')
           .withColumn('max_updated_date', func.max(func.col('updated_date')).over(Window.partitionBy('id')))
           .filter(func.col('updated_date')==func.col('max_updated_date'))
          )

# Principal higher ed institutions

In [3]:
fr_esr = spark.read.format('csv').option("delimiter", ";").option("header","true").load('file:\\' + scanR_path + '\\fr-esr-principaux-etablissements-enseignement-superieur.csv')

In [4]:
fr_esr.printSchema()

root
 |-- identifiant interne: string (nullable = true)
 |-- libellé: string (nullable = true)
 |-- nom court: string (nullable = true)
 |-- sigle: string (nullable = true)
 |-- type d'établissement: string (nullable = true)
 |-- typologie d'universités et assimilés: string (nullable = true)
 |-- secteur d'établissement: string (nullable = true)
 |-- vague contractuelle: string (nullable = true)
 |-- localisation: string (nullable = true)
 |-- site internet: string (nullable = true)
 |-- géolocalisation: string (nullable = true)
 |-- uai - identifiant: string (nullable = true)
 |-- anciens codes uai: string (nullable = true)
 |-- siret: string (nullable = true)
 |-- siren: string (nullable = true)
 |-- rna: string (nullable = true)
 |-- Identifiant wikidata: string (nullable = true)
 |-- Elément wikidata: string (nullable = true)
 |-- identifiant_idref: string (nullable = true)
 |-- Identifiant ETER: string (nullable = true)
 |-- Identifiant ROR: string (nullable = true)
 |-- Elément R

In [5]:
fr_esr = (fr_esr
          .select(func.col('libellé').alias('name'), func.col('nom court').alias('short_name'),
                  func.col('identifiant_idref').alias('idref'), func.col('Identifiant ROR').alias('ror'),
                  func.col('Identifiant wikidata').alias('wikidata'), func.col("type d'établissement").alias('type'),
                  func.col('uai - identifiant').alias('uai'),
                 func.col("secteur d'établissement").alias('secteur'))
          .distinct()
         )
fr_esr.show()
          

+--------------------+--------------------+---------+---------+---------+-------------------+-----------------+-------+
|                name|          short_name|    idref|      ror| wikidata|               type|              uai|secteur|
+--------------------+--------------------+---------+---------+---------+-------------------+-----------------+-------+
|École nationale s...|   ENS Louis Lumière|030067618|003hyqr30|  Q273466|              École|         0932066R| public|
|École des hautes ...|           HEC Paris|027321118|0423jsj19|  Q273535|              École|         0783054W|  privé|
|École supérieure ...|         ESITC Paris|086229834|037mk9523| Q3578412|              École|         0941954N|  privé|
|Institut d'économ...|               IÉSEG|030507200|04jqeag92| Q3156522|              École|         0593202K|  privé|
|École supérieure ...| ÉSAM Caen/Cherbourg|174791534|00c62es15| Q3578397|              École|         0141432F| public|
|    Sciences Po Lyon|         IEP de Ly

In [6]:
fr_esr.groupBy('type').count().show()

+-------------------+-----+
|               type|count|
+-------------------+-----+
|         Université|   65|
|Grand établissement|   25|
|              École|  146|
|Autre établissement|   10|
+-------------------+-----+



In [6]:
merged_by_ror = (df_inst
                 .select('id', 'ror')
                 .join(fr_esr
                       .withColumn('ror', func.concat(func.lit('https://ror.org/'), func.col('ror'))), on = 'ror', how ='inner')
                 .select(func.regexp_replace(func.col('id'), 'https://openalex.org/', '').alias('inst_id'),'idref','uai','name','ror','type',"secteur")
                 .distinct()
                )
merged_by_ror.count()

215

In [8]:
merged_by_ror.show()

+-----------+---------+-----------------+--------------------+--------------------+-------------------+-------+
|    inst_id|    idref|              uai|                name|                 ror|               type|secteur|
+-----------+---------+-----------------+--------------------+--------------------+-------------------+-------+
|I4210118719|027976122|0783664J;0921929X|École supérieure ...|https://ror.org/0...|              École|  privé|
|I4210107334|026408139|         0753667D|Ecole Nationale S...|https://ror.org/0...|              École| public|
|I4387155629|026386283|         0130221V|     Sciences Po Aix|https://ror.org/0...|Autre établissement| public|
| I180375564|120796899|         0290119X|École nationale d...|https://ror.org/0...|              École| public|
| I138816502|029473284|         7200664J|Université de Cor...|https://ror.org/0...|         Université| public|
| I181827316|026582481|         0590344D|Institut catholiq...|https://ror.org/0...|              École| 

In [9]:
merged_by_ror.groupBy('type').count().show()

+-------------------+-----+
|               type|count|
+-------------------+-----+
|         Université|   61|
|Grand établissement|   27|
|              École|  117|
|Autre établissement|   10|
+-------------------+-----+



In [12]:
merged_by_ror.filter(func.col('inst_id')=="I4392021178").show()

+-------+-----+---+----+---+----+-------+
|inst_id|idref|uai|name|ror|type|secteur|
+-------+-----+---+----+---+----+-------+
+-------+-----+---+----+---+----+-------+



In [10]:
df_inst.filter(func.col('id').contains("I4403386658")).show()

+-----------------------+--------------+------------+----------+--------------+------------+------------+---------------------+-------------------------+---+------------+---+---+-------------------+---------+-------------+---------------+-------+------------+-----+---+-------------+-----------+------+----+-------+-------+------------+-------------+-----------+----------+----------------+
|associated_institutions|cited_by_count|country_code|country_id|counts_by_year|created_date|display_name|display_name_acronyms|display_name_alternatives|geo|homepage_url| id|ids|image_thumbnail_url|image_url|international|is_super_system|lineage|repositories|roles|ror|summary_stats|topic_share|topics|type|type_id|updated|updated_date|works_api_url|works_count|x_concepts|max_updated_date|
+-----------------------+--------------+------------+----------+--------------+------------+------------+---------------------+-------------------------+---+------------+---+---+-------------------+---------+----------

In [11]:
#merge = (inst.withColumn('to_join',
#                                           func.regexp_replace(func.lower(func.col('name')),       " de| la| d'| l'| des| du| le| les",'')
#                )
#      .withColumn('to_join', func.regexp_replace(func.col('to_join'), 'é|è', 'e'))
#.join(fr_esr
#      .withColumn('to_join', 
#                  func.explode(func.array([func.regexp_replace(func.lower(func.col('name')),       " de| la| d'| l'| des| du| le| les",''), 
#                                           func.regexp_replace(func.lower(func.col('short_name')), " de| la| d'| l'| des| du| le| les",'')])))
#      .withColumn('to_join', func.regexp_replace(func.col('to_join'), 'é|è', 'e'))
#      , on = 'to_join', how ='inner'))
#merge.count()

In [15]:
fr_esr.count()

246

In [7]:
sorted_by_hand = { #'IAE Paris - Sorbonne Business School':
                   'Université de Mayotte':"I4403928291",
               "École d'ingénieurs des sciences aérospatiales" :"I58581550",
    'Université de technologie Tarbes Occitanie Pyrénées':"I119865316",
    #"Institut régional du travail social de Champagne-Ardennes"
    "Université Bourgogne Europe":"I177064439",
    "Université de Montpellier Paul-Valéry":'I208215962',
    #Institut national du service public
    "Nîmes université":"I66456633",
    #Y SCHOOLS
    "École nationale supérieure de paysage":"I4387154791",
    #Brest Business School
    #École supérieure de fonderie et de forge
    #Institut national des sciences et techniques nucléaires
    #Université Marie et Louis Pasteur
    "Université de Toulouse": "I17866349",
    #Institut supérieur du commerce de Paris,
    #École spéciale d'architecture
    #Institut supérieur de l'électronique et du numérique de Brest
    #Centre national des arts du cirque
    #Institut des relations publiques et de la communication d’Angers
    #ESIEE-IT
    #Institut français de la mode 
    #École nationale supérieure des arts et techniques du théâtre
    "Institut supérieur du bâtiment et des travaux publics": 'I4210160645',
    #Centre des hautes études de la construction
    #École supérieure d'art et de design d'Amiens
    "Université Jean Monnet":"I86767153",
    #Centre de ressources, d'expertise et de performance sportives de Toulouse 
    "École nationale supérieure du pétrole et des moteurs":"I4403386658",
    "Institut protestant de théologie":"I4210121454",
    "Institut supérieur international du parfum, de la cosmétique et de l'aromatique alimentaire":"I4387154221",
    "ICN Business School": "I4400600934",
    "École nationale supérieure des métiers de l'image et du son":'I4387156185',
    #Institut d'ingénierie informatique de Limoges
    "École de management Léonard De Vinci":"I4210121459"
    
                }

In [8]:
print([test[1]for test in sorted_by_hand.items()])

['I4403928291', 'I58581550', 'I119865316', 'I177064439', 'I208215962', 'I66456633', 'I4387154791', 'I17866349', 'I4210160645', 'I86767153', 'I4403386658', 'I4210121454', 'I4387154221', 'I4400600934', 'I4387156185', 'I4210121459']


In [9]:
expr_merge_hand = 'CASE WHEN ' + ' WHEN '.join(['name = "' + el[0] + '" THEN "' + el[1] + '"' for el in sorted_by_hand.items()] )+ ' END'
expr_merge_hand                  

'CASE WHEN name = "Université de Mayotte" THEN "I4403928291" WHEN name = "École d\'ingénieurs des sciences aérospatiales" THEN "I58581550" WHEN name = "Université de technologie Tarbes Occitanie Pyrénées" THEN "I119865316" WHEN name = "Université Bourgogne Europe" THEN "I177064439" WHEN name = "Université de Montpellier Paul-Valéry" THEN "I208215962" WHEN name = "Nîmes université" THEN "I66456633" WHEN name = "École nationale supérieure de paysage" THEN "I4387154791" WHEN name = "Université de Toulouse" THEN "I17866349" WHEN name = "Institut supérieur du bâtiment et des travaux publics" THEN "I4210160645" WHEN name = "Université Jean Monnet" THEN "I86767153" WHEN name = "École nationale supérieure du pétrole et des moteurs" THEN "I4403386658" WHEN name = "Institut protestant de théologie" THEN "I4210121454" WHEN name = "Institut supérieur international du parfum, de la cosmétique et de l\'aromatique alimentaire" THEN "I4387154221" WHEN name = "ICN Business School" THEN "I4400600934" WH

In [10]:
merged_by_hand = (fr_esr
                  .withColumn('inst_id', func.expr(expr_merge_hand))
                  .filter(func.col('inst_id').isNotNull())
                 )
merged_by_hand.show()

+--------------------+----------+---------+---------+----------+----------+-----------------+-------+-----------+
|                name|short_name|    idref|      ror|  wikidata|      type|              uai|secteur|    inst_id|
+--------------------+----------+---------+---------+----------+----------+-----------------+-------+-----------+
|Université Jean M...|      NULL|     NULL|     NULL|      NULL|Université|         0422388T| public|  I86767153|
|Institut supérieu...|    ISIPCA|028549783|     NULL|  Q3152629|     École|0781110H;0783690M|  privé|I4387154221|
|Université Bourgo...|      NULL|     NULL|     NULL|      NULL|Université|         0212296G| public| I177064439|
|École nationale s...|IFP school|026375303|     NULL|  Q2732297|     École|         0920815L| public|I4403386658|
|Université de Tou...|      NULL|     NULL|     NULL|      NULL|Université|         0313218E| public|  I17866349|
|Université de Mon...|      NULL|     NULL|     NULL|      NULL|Université|         0342

In [11]:
merged = (merged_by_ror
          .select('inst_id', 'idref','uai', 'name', 'type','secteur')
          .unionAll(merged_by_hand.select('inst_id', 'idref','uai','name', 'type','secteur'))
          .distinct()
         )
merged.show()

+-----------+---------+-----------------+--------------------+-------------------+-------+
|    inst_id|    idref|              uai|                name|               type|secteur|
+-----------+---------+-----------------+--------------------+-------------------+-------+
|  I88467170|030820529|         0911975C|Université d'Évry...|         Université| public|
| I141576021|031308570|         0762762P|Université Le Hav...|         Université| public|
|I4210152422|026403994|         0311383K|Université Toulou...|         Université| public|
|I4210144611|026436922|         1270009V|École française d...|Autre établissement| public|
| I112936343|033894221|         0690187D|       Centrale Lyon|              École| public|
|I4210108471|026404435|         0720916E|  Le Mans Université|         Université| public|
|I4210103111|133015572|         0310150V|École nationale s...|              École| public|
|I4210150352|136233902|         0693364G|Institut textile ...|              École|  privé|

In [12]:
merged.count()

231

In [47]:
idex_df = (spark.createDataFrame([[key, value] for key,value in idex_dict.items()], 
                                 schema =StructType([StructField('idex',StringType(),True),  
                                        StructField('inst_id',ArrayType(StringType()),True)]) )
           .join(spark.createDataFrame([[key, value] for key,value in idex_dict_dates.items()], 
                                 schema =StructType([StructField('idex',StringType(),True),  
                                        StructField('idex_date',IntegerType(),True)]) ),
                 on = 'idex', how ='left')
           .join(spark.createDataFrame([[key, value] for key,value in idex_dict_dotation.items()], 
                                 schema =StructType([StructField('idex',StringType(),True),  
                                        StructField('idex_dotation',IntegerType(),True)]) ),
                 on = 'idex', how ='left')
          )

idex_df.cache()
idex_df.show()

+--------------------+--------------------+---------+-------------+
|                idex|             inst_id|idex_date|idex_dotation|
+--------------------+--------------------+---------+-------------+
|  idex_aix_marseille|[I21491767, I4210...|     2012|     87785947|
|   idex_annulee_lyon|[I112936343, I203...|     2017|    123694688|
|          isite_next|[I4412460051, I97...|     2017|     37034102|
|     isite_cap_20_25|[I198244214, I222...|     2017|     49328947|
|     idex_strasbourg|         [I68947357]|     2011|     86361781|
|           idex_muse|[I19894307, I4210...|     2017|     84018666|
|       idex_bordeaux|[I3130945570, I42...|     2011|     92201049|
|           idex_daum|         [I90183372]|     2014|      5026000|
|     idex_paris_cite|[I204730241, I148...|     2018|     93979522|
|   idex_paris_saclay|[I195731000, I222...|     2012|    172882291|
|idex_paris_cite_2012|[I204730241, I182...|     2012|     83617599|
|  idex_sorbonne_univ|[I39804081, I4210...|     

In [48]:
fused_df = (spark.createDataFrame([[key, value] for key,value in recoding_fusion.items()], 
                                 schema =StructType([StructField('inst_id',StringType(),True),  
                                        StructField('fused_inst_id',StringType(),True)]) )
           .join(spark.createDataFrame([[key, value] for key,value in fusion_dates.items()], 
                                 schema =StructType([StructField('inst_id',StringType(),True),  
                                        StructField('fusion_date',IntegerType(),True)]) ),
                 on = 'inst_id', how ='left')
          )

fused_df.cache()
fused_df.show()

+-----------+-------------+-----------+
|    inst_id|fused_inst_id|fusion_date|
+-----------+-------------+-----------+
|I4387155307|  I2279609970|       2017|
| I102475099|  I4210107720|       2015|
|I4387156284|   I204730241|       2019|
|I4396570533|    I15057530|       2014|
|I4412460446|   I201841394|       2019|
|   I7171862|  I2279609970|       2017|
|I4387154463|  I4210154111|       2019|
|I4387154560|  I2279609970|       2017|
|  I88467170|   I277688954|       NULL|
| I118618916|   I899635006|       2016|
|I2800379142|  I4210154111|       2019|
|  I90843659|    I90843659|       NULL|
|I4210103002|   I198244214|       2017|
|I4387154756|    I19894307|       2014|
|I4412460119|   I198244214|       2017|
|  I59807433|  I2279609970|       2017|
| I184646667|    I39804081|       2017|
|I4387154366|   I204730241|       2019|
| I177483745|   I899635006|       2016|
|I3123023596|    I56067802|       2023|
+-----------+-------------+-----------+
only showing top 20 rows



In [50]:
expr_rce = "CASE WHEN " + ' WHEN '.join([ 'inst_id = "' + item[0]   + '" THEN "' + str(item[1]) +'"' for item in acces_rce.items()]) + ' ELSE  "0" END'
expr_rce

'CASE WHEN inst_id = "I21491767" THEN "2012" WHEN inst_id = "I4387155829" THEN "2009" WHEN inst_id = "I4210142324" THEN "2009" WHEN inst_id = "I4412460347" THEN "2009" WHEN inst_id = "I138816502" THEN "2009" WHEN inst_id = "I65806277" THEN "2009" WHEN inst_id = "I100532134" THEN "2009" WHEN inst_id = "I4387154463" THEN "2009" WHEN inst_id = "I4387153574" THEN "2009" WHEN inst_id = "I2800745255" THEN "2009" WHEN inst_id = "I90183372" THEN "2012" WHEN inst_id = "I4387156284" THEN "2009" WHEN inst_id = "I4387154366" THEN "2009" WHEN inst_id = "I78744979" THEN "2009" WHEN inst_id = "I86767153" THEN "2009" WHEN inst_id = "I68947357" THEN "2009" WHEN inst_id = "I3131550300" THEN "2009" WHEN inst_id = "I140494188" THEN "2009" WHEN inst_id = "I4396570539" THEN "2010" WHEN inst_id = "I113428412" THEN "2010" WHEN inst_id = "I134560555" THEN "2010" WHEN inst_id = "I110017253" THEN "2010" WHEN inst_id = "I4210156171" THEN "2010" WHEN inst_id = "I195731000" THEN "2010" WHEN inst_id = "I4400573212" 

In [51]:
merged = (merged
          .withColumn('acces_rce', func.expr(expr_rce))
          .join(idex_df
                .withColumn('inst_id', func.explode(func.col('inst_id')))
                .groupBy('inst_id')
                .agg(func.collect_set('idex').alias('idex'),
                     func.min('idex_date').alias('date_first_idex'),
                     func.sum('idex_dotation').alias('total_idex_dotation'))
                , on = 'inst_id', how = 'left')
          .join(fused_df, on = 'inst_id', how = 'left')
         )
merged.count()
merged.show()

+-----------+---------+-----------------+--------------------+-------------------+-------+---------+--------------------+---------------+-------------------+-------------+-----------+
|    inst_id|    idref|              uai|                name|               type|secteur|acces_rce|                idex|date_first_idex|total_idex_dotation|fused_inst_id|fusion_date|
+-----------+---------+-----------------+--------------------+-------------------+-------+---------+--------------------+---------------+-------------------+-------------+-----------+
|  I88467170|030820529|         0911975C|Université d'Évry...|         Université| public|     2011| [idex_paris_saclay]|           2012|          172882291|   I277688954|       NULL|
| I141576021|031308570|         0762762P|Université Le Hav...|         Université| public|     2011|                NULL|           NULL|               NULL|         NULL|       NULL|
|I4210152422|026403994|         0311383K|Université Toulou...|         Universit

In [52]:
merged.write.mode('overwrite').parquet('file:\\' + save_path + 'french_inst_type_w_idex.parquet')

In [17]:
merged= spark.read.parquet('file:\\' + save_path + 'french_inst_type_w_idex.parquet')

In [5]:
df_budget = spark.read.format('json').load('file:\\' + scanR_path + '\\fr-esr-operateurs-indicateurs-financiers.json')

In [6]:
df_budget.show()

+-------------+------------------------------+---------------------------------+----------------------------+-------------------------+------------------+----------------------------+----------------------------------+-------------------------+--------------------------+--------------------------------------+--------------------+------------------------------------------+------------------------------------------+----------------+--------------------------------------+------------------+------------------+-----------------+---------------+---------------------------------------------+--------------------+--------------------+------------------------+--------------------------------+--------+---------------------------------------------------+-----------------------------+------------------------------------------+---------------------------------------+----+------------------+--------------------+-------------------------------+----------------------------------------+-----------------

# Get all institutions from OpenAlex

In [53]:
french_inst_type_w_idex = spark.read.parquet('file:\\' + save_path + 'french_inst_type_w_idex.parquet')

In [ ]:
french_inst_type_w_idex

In [57]:
type_index = {
        row['type'] : row['inst_id'] for row in french_inst_type_w_idex.groupBy('type').agg(func.collect_set('inst_id').alias('inst_id')).collect()
}
secteur_index = {
        row['secteur'] : row['inst_id'] for row in french_inst_type_w_idex.groupBy('secteur').agg(func.collect_set('inst_id').alias('inst_id')).collect()
}
expr_type  = "CASE WHEN " + ' WHEN '.join([( "inst_id IN ('" + "' , '".join(item[1])  + "')" + 
                                                     " OR arrays_overlap(parent.parent_id, array('" + "','".join(item[1]) + "'))"  +
                                                     " THEN '" + item[0] +"'") for item in type_index.items()]) + " ELSE  'undefined' END"
expr_secteur  = "CASE WHEN " + ' WHEN '.join([( "inst_id IN ('" + "' , '".join(item[1])  + "')" + 
                                                     " OR arrays_overlap(parent.parent_id, array('" + "','".join(item[1]) + "'))"  +
                                                     " THEN '" + item[0] +"'") for item in secteur_index.items()]) + " ELSE  'undefined' END"

In [54]:
inst = (df_inst
        .filter( (func.col('geo.country_code').isin(["FR",'GP','GF','NC','RE','MQ','YT','PF','BL','MF','TF','WF'] )) |
                (func.col('country_code').isin(["FR",'GP','GF','NC','RE','MQ','YT','PF','BL','MF','TF','WF'] ) ) |
                (func.col('geo.country')=="France" ) |
              (  (func.col('country_code').isNull()) & (func.col('geo.country_code').isNull()) &
                  (func.array_distinct(func.col('associated_institutions.country_code')) == func.array([func.lit('FR')]))
              )
              | (func.col('homepage_url').rlike("\.fr$|\.fr/"))
                              | ( (func.col('geo.country_code').isNull() )& (func.col('homepage_url').rlike("\.fr$|\.fr/")))
                              | ( (func.col('geo.country_code').isNull() )& (func.array_distinct(func.col('associated_institutions.country_code')) == func.array([func.lit('FR')])))
               )
        .select(func.regexp_replace(func.col('id'), 'https://openalex.org/', '').alias('inst_id'), func.col('display_name').alias('name'),
                'type', 'geo.city','associated_institutions','topic_share','homepage_url')

        .withColumn('topic_share', func.explode_outer(func.transform(func.col('topic_share'), lambda x: func.struct(x.field.display_name.alias('field'), x.value.alias('value')))))
        .groupBy("inst_id", "name",
                'type', 'city','associated_institutions', 'topic_share.field','homepage_url')
        .agg(func.sum('topic_share.value').alias('value'))
        .withColumn('max_share', func.max(func.col('value')).over(Window.partitionBy('inst_id')))
        .filter( (func.col('value')>=0.5*func.col('max_share')) | (func.col('max_share').isNull()))
        .withColumn('topic_share', func.collect_set(func.struct(func.col('field').alias('field'), func.col('value').alias('share'))).over(Window.partitionBy('inst_id')))
        .withColumn('main_topic', func.concat_ws(',', func.array_except(
            func.collect_set( func.when(func.col('value')==func.col('max_share'), func.col('field'))).over(Window.partitionBy('inst_id')), func.array([func.lit(None)]) )))
        .drop('max_share','field','value')
        .distinct()
        .withColumn('parent', func.array_except(func.transform(func.col('associated_institutions'), lambda x: func.when(x.relationship == 'parent', x)) , func.array([func.lit(None)]) ))
        .withColumn('parent', func.transform(func.col('parent'), lambda x : func.struct(func.regexp_replace(x.id, 'https://openalex.org/', '').alias('parent_id'),
                      x.display_name.alias('parent_name'),
                      x.type.alias('parent_type'))))
        .drop('associated_institutions')
        .withColumn('uni_pub', 
                  (  ( ( func.col('type') == "education")
                    & (func.lower(func.col('name')).contains('universit'))
                    & ~(func.lower(func.col('name')).contains('catholi'))
                     & ~ (func.col('inst_id').isin(list_non_fr_non_uni_pub))
                    )
                        |
                    (
                    (func.array_contains(func.col('parent.parent_type'), 'education'))
                    & ~(func.arrays_overlap(func.col('parent.parent_id'),func.array([func.lit(fused_uni) for fused_uni in list_non_fr_non_uni_pub])))
                    & (func.array_max(func.transform(func.col('parent.parent_name'), lambda x : func.when(func.lower(x).contains('universit'), 1))) ==1)
                    & (func.array_max(func.transform(func.col('parent.parent_name'), lambda x : func.when(func.lower(x).contains('catholic'), 1)
                                                    .otherwise(0))) ==0)
                    )
                    ).cast('int')
                   )
        .fillna(0, subset = ['uni_pub'])
       )
cnrs_descendants = [row[0] for row in inst.filter((func.array_contains(func.col('parent.parent_id'),'I1294671590')
                                                   | (func.col('inst_id')=="I1294671590") )
                                                  | (func.col('homepage_url').contains('cnrs'))
                                                 ).select('inst_id').collect()
                   ] + ['I1294671590', 'I57995698']

inst = (inst
        .withColumn('cnrs', ( (func.col('inst_id').isin(cnrs_descendants) )
                             | (func.arrays_overlap(func.col('parent.parent_id'), func.array( [func.lit(el) for el in cnrs_descendants] )))
                             | (func.array_max(func.transform(func.col('parent.parent_name'), lambda x: x.contains('CNRS').cast('int'))) ==1)
                            ).cast('int')
                   )
        .fillna(0, subset = ['cnrs'])
        .withColumn('parent', func.when( (func.col('parent').isNull() ) | (func.col('parent') == func.array([])), func.array(func.struct(func.col('inst_id').alias('parent_id'),
                                                                                           func.col('name').alias('parent_name'),
                                                                                           func.col('type').alias('parent_type'))))
                              .otherwise(func.col('parent'))
        
       )
             #     .withColumn('parent',func.to_json(func.col('parent')))
        
       )
inst.cache()
inst.show(truncate = False)

+-----------+-------------------------------------------------------------------------------+----------+--------------+---------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+----+
|inst_id    |name                                                                           |type      |city          |homepage_url                                             |topic_share                                                                

In [55]:
inst.count() #  5106 # 4972


4972

In [58]:
inst = (inst
          .withColumn('acces_rce', func.expr(expr_rce))
          .join(idex_df
                .withColumn('inst_id', func.explode(func.col('inst_id')))
                .groupBy('inst_id')
                .agg(func.collect_set('idex').alias('idex'),
                     func.min('idex_date').alias('date_first_idex'),
                     func.sum('idex_dotation').alias('total_idex_dotation'))
                , on = 'inst_id', how = 'left')
          .join(fused_df, on = 'inst_id', how = 'left')
        .withColumn('type_fr', func.expr(expr_type))
        .withColumn('type_fr', func.when(func.col('type_fr')!='undefined', func.col('type_fr'))
                    .when(func.col('uni_pub')==1, "Université")
                    .when((func.col('uni_pub')==0) & (func.col('type')=="education"), "École")
                    .when((func.col('name').rlike('École')), "École")
                    .when(func.col('type')=="company", "Firme")
                    .otherwise('undefined')
                   )
        .withColumn('secteur', func.expr(expr_secteur))
        .withColumn('secteur', func.when(func.col('secteur')!='undefined', func.col('secteur'))
                    .when(func.lower(func.col('name')).rlike("de france|établissement français|public|publique"),'public')
                    .when(func.lower(func.col('name')).rlike('privé'), 'privé')
                    .when( (func.col('uni_pub')==1) | (func.col('type')=="government"), "public")
                    .when(func.col('type').isin(["company","nonprofit"]), "privé")
                    .when((func.col('type')=="facility") & 
                          (func.lower(func.col('name')).rlike('observatoire|sorbonne|maison des sciences')), "public")

                    .when((func.col('type_fr')=="École") & 
                          (func.lower(func.col('name')).rlike('national|école normale|école supérieure|académie de|lycée|school of economics|chimie paris|télécom|mines|institut universitaire|ens|purpan|institut français|collège|obs')), "public")
                    .when((func.col('type_fr')=="École") , "privé")
                    .when( (func.col('type')=="healthcare")  & ( (func.lower(func.concat_ws(',', func.col('parent.parent_name'))).rlike('niversit|militaire|armée|national'))
                                                              | (func.col('name').contains('Centre Hospitalier')) ) , "public")
                    .when( (func.col('type')=="healthcare")  &(func.lower(func.concat_ws(',', func.col('parent.parent_name'))).rlike('clinique')), "privé")

                    .when( (func.concat_ws(',', func.col('parent.parent_name')).rlike('National|Publi|Ministère'))  | (func.concat_ws(',', func.col('parent.parent_type')).rlike('government')), "public")
                    .when(func.col('name').rlike('Fondation|Association'), 'privé')
                    .otherwise('undefined')
                   )
       )
inst.show(truncate = False)

+-----------+-------------------------------------------------------------------------------+----------+--------------+---------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+----+---------+-----------------------+---------------+-------------------+-------------+-----------+----------+---------+
|inst_id    |name                                                                           |type      |city          |homepage_url    

In [59]:
inst.write.mode('overwrite').parquet('file:\\' + save_path + 'inst_fr.parquet')

# Classify all public labs

In [10]:
inst = spark.read.parquet('file:\\' + save_path + 'inst_fr.parquet')

In [11]:
inst.filter(func.col('inst_id') == "I4392021178").show(truncate = False) #2308

+-----------+-------------------------------------------------------------------------------------+--------+----+-------------------+--------------+----------+----------------------------------------------------------------------------------------------------------------+-----+-------+----+-------+---------+---------+
|inst_id    |name                                                                                 |type    |city|homepage_url       |topic_share   |main_topic|parent                                                                                                          |fused|uni_pub|cnrs|idex   |type_fr  |secteur  |
+-----------+-------------------------------------------------------------------------------------+--------+----+-------------------+--------------+----------+----------------------------------------------------------------------------------------------------------------+-----+-------+----+-------+---------+---------+
|I4392021178|Unité de recherche interdis

In [28]:
inst.filter(func.col('type') == "facility").count() #2308

2308

In [23]:
inst.filter( (func.col('type') == "facility") & (func.col('type_fr')=='undefined')).count() #1366

1366

In [24]:
inst.filter( (func.col('type') == "facility") & (func.col('secteur')=='public')).count() #1182

1182

In [7]:
public_labs = spark.read.option("multiline","true").json('file:\\' + scanR_path + '\\fr-esr-structures-recherche-publiques-actives.json')


In [10]:
public_labs.groupBy('type_de_structure').count().show()

+--------------------+-----+
|   type_de_structure|count|
+--------------------+-----+
|Fondations de coo...|    6|
|        Unité propre| 1866|
|      Equipe interne|  692|
|         Unité mixte| 1551|
|Groupements d'Int...|    7|
|Unité mixte inter...|   58|
|Collaborations in...|    1|
|Groupements d'Int...|    8|
|Structures Fédéra...|  366|
|      Société privée|    6|
|Groupements de Re...|   64|
+--------------------+-----+



In [15]:
public_labs.filter(func.array_contains(func.col('sigles_des_tutelles'),"CNRS")).groupBy('type_de_structure').count().show()

+--------------------+-----+
|   type_de_structure|count|
+--------------------+-----+
|Fondations de coo...|    2|
|        Unité propre|  210|
|      Equipe interne|  169|
|         Unité mixte| 1046|
|Groupements d'Int...|    1|
|Unité mixte inter...|   56|
|Groupements d'Int...|    2|
|Structures Fédéra...|  189|
|Groupements de Re...|   56|
+--------------------+-----+



In [17]:
public_labs.count()

4625

In [18]:
public_labs.filter(func.array_contains(func.col('sigles_des_tutelles'),"CNRS")).groupBy('type_de_structure').count().show()

+--------------------+-----+
|   type_de_structure|count|
+--------------------+-----+
|Fondations de coo...|    2|
|        Unité propre|  210|
|      Equipe interne|  169|
|         Unité mixte| 1046|
|Groupements d'Int...|    1|
|Unité mixte inter...|   56|
|Groupements d'Int...|    2|
|Structures Fédéra...|  189|
|Groupements de Re...|   56|
+--------------------+-----+



In [23]:
public_labs.filter(func.array_contains(func.col('code_de_nature_de_tutelle'),"UNIV")).groupBy('type_de_structure',
func.array_contains(func.col('sigles_des_tutelles'),"CNRS").alias('has_cnrs')
                                                                                             ).count().show()

+--------------------+--------+-----+
|   type_de_structure|has_cnrs|count|
+--------------------+--------+-----+
|Groupements d'Int...|    true|    1|
|        Unité propre|   false|  951|
|Groupements d'Int...|   false|    2|
|Groupements d'Int...|   false|    4|
|Groupements d'Int...|    true|    1|
|         Unité mixte|    true|  917|
|Structures Fédéra...|    true|  176|
|Unité mixte inter...|   false|    2|
|         Unité mixte|   false|  429|
|Structures Fédéra...|   false|  116|
|      Equipe interne|   false|  222|
|        Unité propre|    true|  126|
|Groupements de Re...|    true|   16|
|Unité mixte inter...|    true|   17|
|      Equipe interne|    true|  157|
|Groupements de Re...|   false|    4|
|Fondations de coo...|    true|    2|
|Fondations de coo...|   false|    2|
+--------------------+--------+-----+



In [ ]:
public_labs.filter(func.col('annee_de_creation').groupBy('type_de_structure').count().show()

In [13]:
public_labs.show()

+--------------------+------------------------+-----------------+-------------------------+---------------------------+-------------------------+--------------------------+-----------------------+-------------------------+-------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+------------------------------+--------------------------+----------------------------+--------------------+---------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             adresse|annee_d_effet_historique|annee_de_creation|code_de_nature_de_tutelle|code_de_niveau_de_structure|code_de_type_de_structure|code_de_type_de_succession|code_de_type_de_tutelle|code_domaine_scientifique|     code_panel_erc

In [9]:
public_labs.groupBy('type_de_succession').count().show()

+--------------------+-----+
|  type_de_succession|count|
+--------------------+-----+
|fusion;fusion;fus...|    1|
|éclatement - fusi...|   18|
|renouvellement;éc...|    1|
|éclatement - fusi...|    1|
|intégration;intég...|    1|
|renouvellement;re...|    1|
|essaimage;fusion;...|    1|
|              fusion|    9|
|éclatement - fusi...|    2|
|éclatement - fusi...|    1|
|                NULL| 3800|
|essaimage;essaima...|    1|
|éclatement - fusi...|    1|
|      renouvellement|  198|
|           essaimage|   32|
|essaimage;intégra...|    1|
|intégration;intég...|    1|
|éclatement - fusi...|    2|
|fusion;fusion;écl...|    1|
|    essaimage-fusion|    2|
+--------------------+-----+
only showing top 20 rows



In [14]:
from nltk.corpus import stopwords
print(stopwords.fileids())
stopwords_dict = {}
for language in stopwords.fileids():
    stopwords_dict[language.capitalize()] = stopwords.words(language)
    
def rm_stopwords(x,y):
    try:
        list_x = x.split(' ')
        list_y = y
        return(' '.join([word for word in list_x if word not in list_y]))
    except:
        return(x)
udf_rm_stopwords = func.udf(rm_stopwords)

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [15]:
import unidecode
def unidecode_debug(x):
    try:
        return unidecode.unidecode(x) if x is not None else None
       
    except:
        return(x)

udf_unidecode = func.udf(unidecode_debug, StringType())

def stemmer(x,y):
    try:
        if y == 'fr' or y == 'French' or y == 'Français':
            output = ' '.join([FrenchStemmer().stem(word) for word in x.split(' ')])
            return(output)
        else:
            output = ' '.join([PorterStemmer().stem(word) for word in x.split(' ')])
            return(output)
    except:
        return(x)
udf_stemmer = func.udf(stemmer)

In [152]:
regex_homepage = r'(?:(?<=[w]{3}\.)|(?<=//)|(?<=[w]{3}6\.))[\w\W]*(?:(?=\.fr)|(?=\.com)|(?=\.org)|(?=\.net)|(?=\.eu)|(?=\.edu)|(?=\.int))'
try:
    matching_url.unpersist()
except:
    print('first run')

matching_url = (inst
                .select('inst_id', 'name', 'homepage_url')
                .withColumn('homepage_url_recoded_oa', func.regexp_extract(func.col('homepage_url'), regex_homepage,0))
                .withColumn('homepage_url_recoded_oa', func.when(func.col('homepage_url_recoded_oa') =='',
                                                              func.regexp_extract(func.col('homepage_url'), 
                                                                                  r'(?:(?<=[w]{3}\.)|(?<=//)|(?<=[w]{3}6\.))[\w\W]*(?=\.[\w\W]{2,})',0))
                            .otherwise(func.col('homepage_url_recoded_oa')))
                
                .withColumn('homepage_url_recoded_oa', func.when(func.col('homepage_url_recoded_oa') =='',
                                                              func.col('homepage_url'))
                            .otherwise(func.col('homepage_url_recoded_oa')))
                .withColumnRenamed('homepage_url', 'url_oa')
                .crossJoin(public_labs
                      .select('numero_national_de_structure', func.col('site_web').alias('homepage_url'),'libelle')
                      .withColumn('homepage_url_recoded_lab', func.regexp_extract(func.col('homepage_url'), regex_homepage,0))
                      .withColumn('homepage_url_recoded_lab', func.when(func.col('homepage_url_recoded_lab') =='',
                                                                    func.regexp_extract(func.col('homepage_url'), 
                                                                                  r'(?:(?<=[w]{3}\.)|(?<=//)|(?<=[w]{3}6\.))[\w\W]*(?=\.[\w\W]{2,})',0))
                                  .otherwise(func.col('homepage_url_recoded_lab')))
                      .withColumn('homepage_url_recoded_lab', func.when(func.col('homepage_url_recoded_lab') =='',
                                                                    func.col('homepage_url'))
                                  .otherwise(func.col('homepage_url_recoded_lab')))
                    .withColumnRenamed('homepage_url', 'url_lab')
                     )
                .filter(func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa'))<=5)
                .withColumn('n_links', func.count(func.col('numero_national_de_structure')).over(Window.partitionBy('numero_national_de_structure')))
                .withColumn('min_levenshtein_s', func.min(func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa')))
                            .over(Window.partitionBy('numero_national_de_structure')))
                .withColumn('min_levenshtein_o', func.min(func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa')))
                            .over(Window.partitionBy('inst_id')))

                .withColumn('homepage_url_recoded_oa_2', func.regexp_extract(func.col('url_oa'), 
                                                                                        r'(?:(?<=[w]{3}\.)|(?<=//)|(?<=[w]{3}6\.))[\w\W]*', 0))
                .withColumn('homepage_url_recoded_lab_2', func.regexp_extract(func.col('url_lab'), 
                                                                                        r'(?:(?<=[w]{3}\.)|(?<=//)|(?<=[w]{3}6\.))[\w\W]*', 0))
                .withColumn('has_perfect_link_lab', func.max( (func.col('homepage_url_recoded_lab')==func.col('homepage_url_recoded_oa')).cast('int')).over(Window.partitionBy('numero_national_de_structure')))
                .withColumn('has_perfect_link_oa', func.max( (func.col('homepage_url_recoded_lab')==func.col('homepage_url_recoded_oa')).cast('int')).over(Window.partitionBy('inst_id')))
                .filter( 
                        (func.col('n_links')==1) 
                    |  (func.col('url_oa')==func.col('url_lab'))
                    | ((func.col('homepage_url_recoded_oa') == func.col('homepage_url_recoded_lab'))
                     &( (func.col('min_levenshtein_s')==func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa')))
                           | (func.col('min_levenshtein_o')==func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa')))
                         )
                      )
                    | (
                         (func.col('homepage_url_recoded_oa_2') == func.col('homepage_url_recoded_lab_2'))
                        &( (func.col('min_levenshtein_s')==func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa')))
                           | (func.col('min_levenshtein_o')==func.levenshtein(func.col('homepage_url_recoded_lab'), func.col('homepage_url_recoded_oa')))
                         )
                    )
                    
                    | (
                         (func.col('homepage_url_recoded_oa') == func.col('homepage_url_recoded_lab'))  
                    &
                        # (    (func.col('has_perfect_link_lab')==0) | ( func.col('has_perfect_link_oa')==0) )
                           #& 
                        (func.levenshtein(func.col('homepage_url_recoded_oa_2'), func.col('homepage_url_recoded_lab_2'))<=3)
                        

                    )
#
                    
                    |
                    
                    (
                        ( func.levenshtein(func.col('url_lab'), func.col('url_oa')) <=3)  #& (func.col('has_perfect_link_lab')+ func.col('has_perfect_link_oa')==0)
                        &( (func.col('min_levenshtein_s')==func.levenshtein(func.col('url_lab'), func.col('url_oa')))
                           | (func.col('min_levenshtein_o')==func.levenshtein(func.col('url_lab'), func.col('url_oa')))
                         )
                        ) 
)

                .distinct()
               )
matching_url.cache()
matching_url.count() #8904

8904

In [218]:
matching_url_2 = (matching_url
                .withColumn('n_links_s', func.count(func.col('inst_id')).over(Window.partitionBy('numero_national_de_structure')))
                .withColumn('n_links_oa', func.count(func.col('numero_national_de_structure')).over(Window.partitionBy('inst_id')))
                .withColumn('homepage_url_recoded_lab_2', func.regexp_replace(func.col('homepage_url_recoded_lab_2'), 'http://www.|https://www.|https://www6.|http://www6.|http://|https://|\?lang=en|\?lang=fr|$|/en',''))
                .withColumn('homepage_url_recoded_oa_2', func.regexp_replace(func.col('homepage_url_recoded_oa_2'), 'http://www.|https://www.|https://www6.|http://www6.|http://|https://|\?lang=en|\?lang=fr|/$|/en',''))
                .withColumn('levenshtein', func.levenshtein(func.col('homepage_url_recoded_lab_2'), func.col('homepage_url_recoded_oa_2')))
                .withColumn('min_levenshtein_s', func.min('levenshtein')
                            .over(Window.partitionBy('numero_national_de_structure')))
                .withColumn('min_levenshtein_o', func.min("levenshtein")
                            .over(Window.partitionBy('inst_id')))

                .filter( ( (func.col('n_links_s')==1) & (func.col('n_links_oa')==1))
                       |
                       ( (func.col('levenshtein')==func.col('min_levenshtein_s'))
                        & (func.col('levenshtein')==func.col('min_levenshtein_o'))
                        & (func.col('levenshtein')<=3)
                       )
                       
                       )
                .withColumn('n_links_s', func.count(func.col('inst_id')).over(Window.partitionBy('numero_national_de_structure')))
                .withColumn('n_links_oa', func.count(func.col('numero_national_de_structure')).over(Window.partitionBy('inst_id')))
                 )
matching_url_2.cache()
matching_url_2.filter(func.col('n_links_oa')<=2).count()
#matching_url_2.groupBy('n_links_s','n_links_oa').count().sort(func.col('count').desc()).show()

1313

In [161]:
public_labs.filter(func.col('numero_national_de_structure')=="201924068H").show()

+--------------------+------------------------+-----------------+-------------------------+---------------------------+-------------------------+--------------------------+-----------------------+-------------------------+--------------+-----------+-------+--------------------+--------------------+------------+--------------+--------------------+------------------+--------------------------+------------------------------+--------------------------+----------------------------+--------------------+---------------------+----------+-------------------+------------------+--------------------+--------------------+--------------------+-----------------+------------------+--------------------+----------------+
|             adresse|annee_d_effet_historique|annee_de_creation|code_de_nature_de_tutelle|code_de_niveau_de_structure|code_de_type_de_structure|code_de_type_de_succession|code_de_type_de_tutelle|code_domaine_scientifique|code_panel_erc|code_postal|commune|domaine_scientifique|         

In [219]:
unmatched = (public_labs
             .join(matching_url_2.filter(func.col('n_links_oa')<=2).select('numero_national_de_structure'), on = 'numero_national_de_structure', how = 'left_anti')
            )
#unmatched.cache()
unmatched.count() #3332

3332

In [21]:
#public_labs.count() #4625

4625

In [220]:
try:
    test_fuzzy_matching.unpersist()
except:
    print('first run')

test_fuzzy_matching = (inst
                       .select('inst_id', 'name', 'city')
                       .withColumn('city', (func.lower(func.col('city'))))
                       .withColumn('city', func.translate(func.col('city'), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"))
                        .withColumn('city', func.regexp_replace(func.col('city'), '-', ' '))
                        .withColumn('name', func.regexp_replace(func.translate(func.lower(func.col('name')), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"), r'[^\w\s\-]', ''))
                       #.withColumn('name', udf_rm_stopwords(func.regexp_replace(udf_unidecode(func.lower(func.col('name'))), r'[^\w\s\-]', ''), func.array([func.lit(x) for x in stopwords_dict['French']])))
                       .crossJoin(unmatched
                             .select('numero_national_de_structure', 'libelle', 'commune')
                             .withColumn('city_s', func.translate(func.lower(func.col('commune')), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"))
                           .withColumn('city_s', func.regexp_replace(func.col('city_s'), r'\scedex|\s\d+', ''))
                           .withColumn('city_s', func.regexp_replace(func.col('city_s'), '-', ' '))

                            .withColumn('libelle', func.regexp_replace(func.translate(func.lower(func.col('libelle')), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"), r'[^\w\s\-]', ''))
                            # .withColumn('libelle', udf_rm_stopwords(func.regexp_replace(udf_unidecode(func.lower(func.col('libelle'))), r'[^\w\s\-]', ''), func.array([func.lit(x) for x in stopwords_dict['French']])))
                             )
                       .withColumn('token_oa', func.split(func.col('name'),' '))
                       .withColumn('token_pub', func.split(func.col('libelle'),' '))
                        .withColumn('n_shared_tokens', func.size(func.array_intersect(func.col('token_oa'), func.col('token_pub') )))
                         .filter((func.col('n_shared_tokens')>0)
                                # & ( (func.col('city')== func.col('city_s'))
                                #    | (func.col('city').isNull())
                                #    | (func.col('city_s').isNull())
                                #   )
                                )
                      )
test_fuzzy_matching.cache()
test_fuzzy_matching.count() #4030144

4030144

In [63]:
test_fuzzy_matching.filter(func.col('numero_national_de_structure').isin(["200815493K",'198821638B'])).show()

+-----------+--------------------+-------------------+----------------------------+--------------------+-------+------+--------------------+--------------------+---------------+
|    inst_id|                name|               city|numero_national_de_structure|             libelle|commune|city_s|            token_oa|           token_pub|n_shared_tokens|
+-----------+--------------------+-------------------+----------------------------+--------------------+-------+------+--------------------+--------------------+---------------+
| I100445878|ecole centrale de...|             nantes|                  201722473H|unite de recherch...|   NULL|  NULL|[ecole, centrale,...|[unite, de, reche...|              1|
|  I11935315|ecole nationale s...|        montpellier|                  201722473H|unite de recherch...|   NULL|  NULL|[ecole, nationale...|[unite, de, reche...|              1|
| I119865316|ecole nationale d...|             tarbes|                  201722473H|unite de recherch...|   NUL

In [48]:
test_fuzzy_matching.filter(func.col('inst_id').isin(["I4392021178"])).show()

+----+-----------+--------------------+----------------------------+--------------------+-------------+--------------------+--------------------+---------------+
|city|    inst_id|                name|numero_national_de_structure|             libelle|      commune|            token_oa|           token_pub|n_shared_tokens|
+----+-----------+--------------------+----------------------------+--------------------+-------------+--------------------+--------------------+---------------+
|caen|I4392021178|unite de recherch...|                  201220890J|unite support cyc...| CAEN CEDEX 5|[unite, de, reche...|[unite, support, ...|              1|
|caen|I4392021178|unite de recherch...|                  200810701B|institut de reche...| CAEN CEDEX 4|[unite, de, reche...|[institut, de, re...|              2|
|caen|I4392021178|unite de recherch...|                  199814135C|equipe de recherc...|   CAEN CEDEX|[unite, de, reche...|[equipe, de, rech...|              5|
|caen|I4392021178|unite de r

In [221]:
from rapidfuzz import fuzz
from pyspark.sql.types import *
udf_ratio = func.udf(fuzz.ratio, DoubleType() )
udf_token_sort = func.udf(fuzz.token_sort_ratio, DoubleType())

test_fuzzy_matching_2  = (test_fuzzy_matching
                          .withColumn('levenshtein', func.levenshtein(func.col('libelle'), func.col('name'))/func.greatest(func.length(func.col('libelle')), func.length(func.col('name'))))
                         # .withColumn('max_length_str', func.greatest(func.length(func.col('libelle')), func.length(func.col('name'))))
                         .withColumn('fuzz_ratio', udf_ratio(func.col('libelle'), func.col('name')))
                         .withColumn('token_sort_ratio', udf_token_sort(func.col('libelle'), func.col('name')))
                         .filter( func.col('token_sort_ratio') >= 66 )
                          .filter(func.col('fuzz_ratio')>=66)
                         .withColumn('max_match_measure_t', func.max(func.col('token_sort_ratio')).over(Window.partitionBy('libelle')))
                         )
test_fuzzy_matching_2.show(truncate = False)

+-----------+---------------------------------------------------------------+-------------------+----------------------------+-----------------------------------------------------------------------------+-------------------------------+-------------------------+------------------------------------------------------------------------+------------------------------------------------------------------------------------------+---------------+-------------------+-----------------+-----------------+-------------------+
|inst_id    |name                                                           |city               |numero_national_de_structure|libelle                                                                      |commune                        |city_s                   |token_oa                                                                |token_pub                                                                                 |n_shared_tokens|levenshtein        |fuzz_ratio       |t

In [222]:
test_fuzzy_matching_2.count() #6826

6810

In [66]:
test_fuzzy_matching_2.filter((func.col('token_sort_ratio') != func.col('max_match_measure_t') )).show(truncate = False)

+-----------+---------------------------------------------------------------+--------------+----------------------------+-----------------------------------------------------------------------+----------------+--------+------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------+-------------------+-----------------+-----------------+-------------------+
|inst_id    |name                                                           |city          |numero_national_de_structure|libelle                                                                |commune         |city_s  |token_oa                                                                |token_pub                                                                          |n_shared_tokens|levenshtein        |fuzz_ratio       |token_sort_ratio |max_match_measure_t|
+-----------+-------------------------------------------------

In [67]:
test_fuzzy_matching_2.filter(func.col('numero_national_de_structure')=='201722473H').show(truncate = False)

+-----------+-------------------------------------------------------------------------------------+----+----------------------------+-------------------------------------------------------------------------------------+-------+------+--------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+---------------+-----------+----------+----------------+-------------------+
|inst_id    |name                                                                                 |city|numero_national_de_structure|libelle                                                                              |commune|city_s|token_oa                                                                                          |token_pub                                                                                         |n_shared_tokens|levenshtein|fuzz_ratio|token_sort_ratio|max

In [223]:
fuzzy_matches = (test_fuzzy_matching_2
                 .filter((func.col('token_sort_ratio') == func.col('max_match_measure_t') )
                        )
                 .withColumn('n_matches', func.count('inst_id').over(Window.partitionBy('numero_national_de_structure')))
                 .filter('n_matches=1')
                .withColumn('max_match_measure_t', func.max(func.col('token_sort_ratio')).over(Window.partitionBy('inst_id')))
                 .withColumn('n_city', func.size(func.collect_set(func.col('city_s')).over(Window.partitionBy('inst_id') )))
                 .filter((func.col('token_sort_ratio') == func.col('max_match_measure_t') )
                         & ( (func.col('n_city')==1) | (func.col('city')==func.col('city_s') )
                             |  ( func.col('city').isNull() ) |  (func.col('city_s').isNull() ) 
                         )
                        )
                )
fuzzy_matches.cache()
fuzzy_matches.count() #960

949

In [224]:
fuzzy_matches.groupBy('inst_id').count().filter('count>1').show()

+-----------+-----+
|    inst_id|count|
+-----------+-----+
|I4210127988|    2|
|I4210113963|    2|
|I4210112285|    2|
|I4210161954|    2|
|I4210100775|    2|
+-----------+-----+



In [225]:
fuzzy_matches.filter(func.col('inst_id')=="I4210100775").show(truncate = False)

+-----------+-------------------------------------------------+----+----------------------------+-----------------------------------------------------+-------------+------+---------------------------------------------------------+--------------------------------------------------------------+---------------+-------------------+-----------------+-----------------+-------------------+---------+------+
|inst_id    |name                                             |city|numero_national_de_structure|libelle                                              |commune      |city_s|token_oa                                                 |token_pub                                                     |n_shared_tokens|levenshtein        |fuzz_ratio       |token_sort_ratio |max_match_measure_t|n_matches|n_city|
+-----------+-------------------------------------------------+----+----------------------------+-----------------------------------------------------+-------------+------+----------------------

In [227]:
matches = (matching_url_2
           .select('numero_national_de_structure', 'inst_id')
           .unionAll(fuzzy_matches.filter(~func.col('inst_id').isin(['I4210127988',"I4210161954"])).select('numero_national_de_structure', 'inst_id'))
          )
matches.count() #2386

2386

In [48]:
matches.filter(func.col('inst_id')=="I4392021178").show()

+----------------------------+-----------+
|numero_national_de_structure|    inst_id|
+----------------------------+-----------+
|                  201722473H|I4392021178|
+----------------------------+-----------+



In [228]:
unmatched.unpersist()
unmatched = (public_labs
             .join(matches.select('numero_national_de_structure'), on = 'numero_national_de_structure', how = 'left_anti')
            )
unmatched.cache()
unmatched.count() #2289

2289

In [230]:
unmatched.printSchema()

root
 |-- numero_national_de_structure: string (nullable = true)
 |-- adresse: string (nullable = true)
 |-- annee_d_effet_historique: string (nullable = true)
 |-- annee_de_creation: string (nullable = true)
 |-- code_de_nature_de_tutelle: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- code_de_niveau_de_structure: string (nullable = true)
 |-- code_de_type_de_structure: string (nullable = true)
 |-- code_de_type_de_succession: string (nullable = true)
 |-- code_de_type_de_tutelle: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- code_domaine_scientifique: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- code_panel_erc: string (nullable = true)
 |-- code_postal: string (nullable = true)
 |-- commune: string (nullable = true)
 |-- domaine_scientifique: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fiche_rnsr: string (nullable = true)
 |-- label_numero: array (nullable = t

In [51]:
unmatched.filter(func.col('annee_de_creation')<2021).count() #1831

1831

In [93]:
public_labs.filter(func.col('annee_de_creation')<2021).count() #4211

4211

In [87]:
matches.filter(func.col('numero_national_de_structure').isin(["200815493K",'198821638B'])).show()

+----------------------------+-----------+
|numero_national_de_structure|    inst_id|
+----------------------------+-----------+
|                  198821638B|I4387155715|
|                  200815493K|I4392021105|
+----------------------------+-----------+



In [231]:
unmatched.select('libelle','numero_national_de_structure','annee_de_creation','site_web').show(40,truncate = False)

+-----------------------------------------------------------------------------------------------------------------+----------------------------+-----------------+-----------------------------------------------------------------------------------------+
|libelle                                                                                                          |numero_national_de_structure|annee_de_creation|site_web                                                                                 |
+-----------------------------------------------------------------------------------------------------------------+----------------------------+-----------------+-----------------------------------------------------------------------------------------+
|INSTITUT DE CHIMIE DE LYON                                                                                       |200710696A                  |2007             |http://www.iclyon.fr/                                                          

In [97]:
matches.filter(func.col('inst_id')=="I4387154857").show()

+----------------------------+-----------+
|numero_national_de_structure|    inst_id|
+----------------------------+-----------+
|                  202023559A|I4387154857|
+----------------------------+-----------+



In [62]:
inst.filter(func.col('type')=="facility").join(matches.select('inst_id'), on = 'inst_id', how = 'left_anti').select('inst_id','name','city').show(truncate =False) # 537

537

In [30]:
df_inst.filter(func.col('id').contains("I4392021178")).show()

In [105]:
unmatched.filter(func.lower(func.col('libelle')).contains('signaux')).show(truncate =False)

+----------------------------+-------------------------------------------------------+------------------------+-----------------+-------------------------+---------------------------+-------------------------+--------------------------+------------------------+-------------------------+--------------+-----------+---------------------------------+-------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------+------------------------------+------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+------------------------+------------------------------------------------------------------------------------------------------------+------------------------------+--------------------------+-----------------------------------------

In [26]:
unmatched.select('libelle','numero_national_de_structure','annee_de_creation').groupBy('annee_de_creation').count().sort('annee_de_creation').show(100, truncate = False)

+-----------------+-----+
|annee_de_creation|count|
+-----------------+-----+
|1800             |2    |
|1909             |1    |
|1953             |1    |
|1955             |2    |
|1958             |1    |
|1959             |2    |
|1960             |2    |
|1961             |1    |
|1962             |2    |
|1963             |3    |
|1964             |1    |
|1966             |4    |
|1967             |1    |
|1968             |2    |
|1969             |3    |
|1970             |2    |
|1972             |1    |
|1973             |1    |
|1975             |2    |
|1978             |1    |
|1979             |2    |
|1980             |1    |
|1981             |2    |
|1982             |4    |
|1983             |1    |
|1985             |2    |
|1986             |1    |
|1987             |2    |
|1988             |2    |
|1989             |3    |
|1990             |9    |
|1991             |17   |
|1992             |52   |
|1993             |10   |
|1994             |11   |
|1995       

In [129]:
(inst
                       .select('inst_id', 'name', 'city')
                       .withColumn('city', func.translate(func.col('city'), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"))
                       .withColumn('city', func.regexp_extract(func.col('city'),r'^\w*$|^\w*(?=\s)|^[\w\-]*', 0))
                       .withColumn('city', (func.lower(func.col('city'))))
                       .withColumn('city', func.regexp_replace(func.col('city'), 'r\W', ' '))
                        .withColumn('name', func.regexp_replace(func.translate(func.lower(func.col('name')), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"), r'[^\w\s\-]', ''))
                       #.withColumn('name', udf_rm_stopwords(func.regexp_replace(udf_unidecode(func.lower(func.col('name'))), r'[^\w\s\-]', ''), func.array([func.lit(x) for x in stopwords_dict['French']])))
                       .join(unmatched.filter(func.col('numero_national_de_structure')=="200815493K")
                             .select('numero_national_de_structure', 'libelle', 'commune')
                             .withColumn('city', func.translate(func.lower(func.col('commune')), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"))
                             .withColumn('city', func.regexp_extract(func.col('city'),r'^\w*$|^\w*(?=\s)|^[\w\-]*', 0))
                             .withColumn('city', func.regexp_replace(func.col('city'), 'r\W', ' '))
                            .withColumn('libelle', func.regexp_replace(func.translate(func.lower(func.col('libelle')), "éèêëàâäîïôöùûüç", "eeeeaaaiioouuuc"), r'[^\w\s\-]', ''))
                            # .withColumn('libelle', udf_rm_stopwords(func.regexp_replace(udf_unidecode(func.lower(func.col('libelle'))), r'[^\w\s\-]', ''), func.array([func.lit(x) for x in stopwords_dict['French']])))
                             , on = ['city'], how ='inner')
                         .withColumn('n_shared_tokens', func.size(func.array_intersect(func.split(func.col('name'),' '), func.split( func.col('libelle'), ' '))))
.filter(func.col('n_shared_tokens')>1)).collect()

[Row(city='caen', inst_id='I4392021105', name='lettres arts du spectacle langues romanes', numero_national_de_structure='200815493K', libelle='lettres arts du spectacle langues romanes', commune='CAEN CEDEX', n_shared_tokens=6)]

In [182]:
inst.filter(func.concat_ws(',',func.col('parent.parent_id')).contains('I154526488')).select('name',"inst_id").filter(func.lower(func.col('name')).rlike('molécu')).collect()

[Row(name='Laboratoire des Biomolécules', inst_id='I4210165010'),
 Row(name='Institut thématique Bases moléculaires et structurales du vivant', inst_id='I4210094820'),
 Row(name='Institut de génétique et de biologie moléculaire et cellulaire', inst_id='I4210086872')]

In [53]:
public_labs.filter(func.col('numero_national_de_structure')=="200617985C").show(truncate= False)

+--------------------+------------------------+-----------------+-----------------------------+---------------------------+-------------------------+--------------------------+------------------------+-------------------------+--------------+-----------+--------+-------------------------------------------------------------------+--------------------------------------------------------------------------------------------+----------------------------------------------+-----------------------------------+-----------------------------------------------------------------------------------------+-----------------------------+--------------------------+------------------------------+--------------------------+----------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [91]:
matches.filter(func.col('numero_national_de_structure')==('198821638B')).show(truncate = False)

+----------------------------+-------+
|numero_national_de_structure|inst_id|
+----------------------------+-------+
+----------------------------+-------+



In [88]:
(public_labs.filter(func.col('numero_national_de_structure')=="198821638B")
      .select('numero_national_de_structure', 'libelle', 'commune')
                             .withColumn('city', func.regexp_extract(func.col('commune'),r'^\w*$|^\w*\s|^[\w\-]*', 0))
                             .withColumn('city', udf_unidecode(func.lower(func.col('city'))))
                             .withColumn('city', func.regexp_replace(func.col('city'), 'r\W', ' '))
                             .withColumn('libelle', udf_rm_stopwords(func.regexp_replace(udf_unidecode(func.lower(func.col('libelle'))), r'[^\w\s\-]', ''), func.array([func.lit(x) for x in stopwords_dict['French']])))

                             ).collect()

[Row(numero_national_de_structure='198821638B', libelle='centre recherche inegalites sociales', commune='PARIS', city='paris')]

In [46]:
public_labs.groupBy('type_de_structure').count().show()

+--------------------+-----+
|   type_de_structure|count|
+--------------------+-----+
|Fondations de coo...|    6|
|        Unité propre| 1866|
|      Equipe interne|  692|
|         Unité mixte| 1551|
|Groupements d'Int...|    7|
|Unité mixte inter...|   58|
|Collaborations in...|    1|
|Groupements d'Int...|    8|
|Structures Fédéra...|  366|
|      Société privée|    6|
|Groupements de Re...|   64|
+--------------------+-----+



In [50]:
public_labs.filter(func.col('type_de_structure') == "Unité propre").select('libelle').show(truncate= False)

+-----------------------------------------------------------------------------------------------------------------+
|libelle                                                                                                          |
+-----------------------------------------------------------------------------------------------------------------+
|Cerema Direction Territoriale Centre-Est                                                                         |
|CENTRE DE RECHERCHES ANGLOPHONES                                                                                 |
|Institut de Recherches Philosophiques                                                                            |
|CENTRE DE DROIT INTERNATIONAL DE NANTERRE                                                                        |
|Institut Droit Ethique Patrimoine                                                                                |
|LABORATOIRE DE SOCIOLINGUISTIQUE, D'ANTHROPOLOGIE DES PRATIQUES LANGAGI

In [3]:
df_inst.filter(func.col('id').contains('I4392021178')).show(truncate = False)

+-----------------------+--------------+------------+----------+--------------+------------+-------------------------------------------------------------------------------------+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------+-------------------+--------------------------------+-------------------------------------------------------------------------------------+-------------------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
(public_labs
 .withColumn('uai', func.explode_outer(func.col('uai_des_tutelles')))
.join(df_budget.select(func.col('uai_identifiant').alias('uai')).distinct(), on = 'uai', how = 'inner')
.select('numero_national_de_structure').distinct()
).count()

In [42]:
(public_labs
 .withColumn('uai', func.explode_outer(func.col('uai_des_tutelles')))
.join(df_budget.select(func.col('uai_identifiant').alias('uai')).distinct(), on = 'uai', how = 'inner')
.groupBy('numero_national_de_structure').count()
 .groupBy('count').count()
).show()

+-----+-----+
|count|count|
+-----+-----+
|    7|    4|
|   32|    2|
|   43|    1|
|   31|    1|
|   25|    1|
|    6|   29|
|    9|    5|
|   17|    1|
|    5|   31|
|    1| 2339|
|   10|    4|
|    3|  215|
|   37|    1|
|   12|    1|
|    8|    4|
|   11|    3|
|    2|  592|
|    4|   65|
|   13|    1|
|   14|    1|
+-----+-----+
only showing top 20 rows



In [34]:
spark.catalog.clearCache()
spark.stop()